In [8]:
import numpy as np
from scipy.stats import bernoulli
from random import seed
from random import random

In [30]:
def initialize_dist(K):
    beta_distn_reward = []
    beta_distn_cost = []

    for i in range(K):
        beta_distn_reward.append([0.0,0.0])

    for i in range(k):
        beta_distn_cost.append([0.0,0.0])

    return beta_distn_reward, beta_distn_cost

def sample_reward_and_cost(arm_pulled, mu, cost):
    reward_received = bernoulli.rvs(mu[arm_pulled], size=1)[0]
    cost_received = bernoulli.rvs(cost[arm_pulled], size=1)[0]
    return reward_received, cost_received

def update_distn_and_budget(arm_pulled, budget, beta_distn_reward, beta_distn_cost, reward_received, cost_received):
    # cost_records.append(cost_received)
    # reward_records.append(reward_received)
    # arm_pulled_records.append(arm_pulled)
    beta_distn_reward[arm_pulled][0] += reward_received
    beta_distn_reward[arm_pulled][1] += (1-reward_received)
    beta_distn_cost[arm_pulled][0] += cost_received
    beta_distn_cost[arm_pulled][1] += (1-cost_received)
    return beta_distn_reward, beta_distn_cost

def choose_arm(beta_distn_reward, beta_distn_cost, k, budget, mu, cost):
    # reward_records = []
    # arm_pulled_records = []
    # cost_records = []

    sampled_mean_reward = np.array([0]*k, dtype=np.float)
    sampled_mean_cost = np.array([0]*k, dtype=np.float)

    if budget>0:
        for arm in range(k):
            sampled_mean_reward[arm] = np.random.beta(beta_distn_reward[arm][0]+1, beta_distn_reward[arm][1]+1)
            sampled_mean_cost[arm] = np.random.beta(beta_distn_cost[arm][0]+1, beta_distn_cost[arm][1]+1)
        arm_pulled = np.argmax(sampled_mean_reward/sampled_mean_cost)
        #reward_received, cost_received = sample_reward_and_cost(arm_pulled, mu, cost)
        #budget, beta_distn_reward, beta_distn_cost = update_distn_and_budget(arm_pulled, budget, beta_distn_reward, beta_distn_cost, reward_received,cost_received)
    return arm_pulled
    #return reward_received, cost_received, arm_pulled, budget, beta_distn_reward, beta_distn_cost

def compute_best_arm(mu, cost):
    return np.argmax(np.array(mu)/np.array(cost))

In [10]:
def initialize(K, B, min_mu_cost):
    """
    Parameters
    K - Number of arms
    B - Budget
    min_mu_cost - The minimum cost in the bernoulli trial
    Returns
    max_size - 2*B/min_cost
    N - A 2d array of size max_size and arms containing the total number of times an arm has been played at each time step
    X - A 2d array of size max_size and arms containing total reward obtained at each time step
    C - A 2d array of size max_size and arms containing total cost obtained at each time step
    arm_
    """
    max_size = int(2*B/min_mu_cost)
    N = np.zeros((max_size, K))
    X = np.zeros((max_size, K))
    C = np.zeros((max_size, K))
    arm_pulled = np.zeros(max_size, dtype = "int")
    return N,X,C,arm_pulled


def update(N, X, C, t):
    N[t] = N[t-1]
    X[t] = X[t-1]
    C[t] = C[t-1]

In [42]:
def fairness_with_budget_thompson_sampling(K, budget, mu, cost, alpha = None, R = None):
    
    min_mu_cost = min(cost)
    arm_pulled_count, reward_records, cost_records, arm_pulled_records = initialize(K, budget, min_mu_cost)

    t = 0
    beta_distn_reward, beta_distn_cost = initialize_dist(K)

    while budget > 0:
        t += 1
        play_thomp = True
        arm_pulled = -1
        if(alpha is not None and R is not None):
            unfair_arm = []
            unfair_val = []
            for i in range(K):
                if (R[i]*(t-1) - arm_pulled_count[t][i]) > alpha:
                    unfair_arm.append(i)
                    unfair_val.append(r[i]*(t-1) - arm_pulled_count[t][i])

            if unfair_arm:
                play_thomp = False
                arm_pulled = unfair_arm[np.argmax(np.array(unfair_val))]
                
        if play_thomp:
            arm_pulled = choose_arm(beta_distn_reward, beta_distn_cost, K, budget, mu, cost)
        
        update(arm_pulled_count, reward_records, cost_records, t)
        
        reward_received, cost_received = sample_reward_and_cost(arm_pulled, mu, cost)        
        beta_distn_reward, beta_distn_cost = update_distn_and_budget(arm_pulled, budget, beta_distn_reward, 
                                   beta_distn_cost, reward_received, cost_received)
        
        arm_pulled_count[t][arm_pulled] += 1
        budget -= cost_received
        cost_records[t][arm_pulled] += cost_received
        #print(cost_records[t][arm_pulled])
        reward_records[t][arm_pulled] += reward_received
        arm_pulled_records[t] = arm_pulled

    return arm_pulled_count, reward_records, cost_records, arm_pulled_records, t #N,

In [43]:
def compute_regret(C, arm_pulled, cost_means, reward_means, tB):
    """
    Returns 
    regret - an array size of tB+1 containing regret at each round
    #budget - total budget at end of round t
    """
    regret = np.zeros(tB+1)
    reg_sum = np.zeros(tB+1)
    budget_sum = np.zeros(tB+1)
    best_arm = compute_best_arm(reward_means, cost_means)
    #budget = np.zeros(tB+1)
    for i in range(tB+1):
        #budget[i] = np.sum(C[t])
        arm = arm_pulled[i]
        regret[i] = cost_means[arm]*(reward_means[best_arm]/cost_means[best_arm] - reward_means[arm]/cost_means[arm])
        #regret[i] = reward_means[best_arm] - reward_means[arm]
        reg_sum[i] = (reg_sum[i-1] + regret[i]) if i > 0 else regret[i]
        budget_sum[i] = np.sum(C[i])
        
    return regret, reg_sum, budget_sum

In [44]:
cost = [0.71138893, 0.74408936, 0.42790662, 0.81618107, 0.12610527,
       0.57237819, 0.83745534, 0.89507443, 0.92044633, 0.10855726]
mu = [0.53761232, 0.6889405 , 0.13180401, 0.52978279, 0.16823828,
       0.86604129, 0.99220976, 0.74377085, 0.34088901, 0.23490474]

In [45]:
j = 0
k = 10
B = 1000
while(j <= 9):
                 #budget
    # r = np.zeros(k)
    r = np.array([0.05]*k)          #fairness_array for different arms
    alpha = 1e10                       #tolerance_parameter for fairness

    # T = 1000
    #compute_regret(C, arm_pulled, cost_means, reward_means, tB):
    
    arm_pulled_count, reward_records, cost_records, arm_pulled_records, tB = fairness_with_budget_thompson_sampling(k, B, mu, cost)
    regret, reg_sum, budget_sum = compute_regret(cost_records, arm_pulled_records, cost, mu, tB)
    #return N,X,C,arm_pulled,t
    print(reg_sum[-1], tB)
#     regret_record = compute_regret(arm_pulled_records, mu, cost)
    j += 1
#     print (regret_record[-1])
#     print (len(regret_record))


151.561812238776 8058
156.65545607264337 8156
168.73797597292335 8917
197.27143964439094 7880
178.70651969724634 7891
181.69959374208875 7617
267.55832248054645 7360
219.14214901136256 8445
336.17706450714337 6723
708.4772778597873 2560
